In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = ''
df = pd.read_csv(file_path)

# Display the DataFrame for review
df.head()


In [ ]:
import openai
import pandas as pd

# Assuming your API key is already set up
# openai.api_key = "YOUR_OPENAI_API_KEY"

# Sample DataFrame setup for illustration
# Replace this with your actual DataFrame
# df = pd.DataFrame({
#     'posts': ["Post content here", "Another post content"],
#     'substance_use': [1, 0]  # 1 for substance-related, 0 for non-substance
# })

# Filter for substance use posts
substance_use_df = df[df['substance'] == 1].copy()
print(substance_use_df.shape)

# Define the prompt structure as a function
def analyze_post(post_content):
    prompts = [
        {
            "topic": "Family Influence",
            "prompt": f"You will receive a document related to adolescent behaviors. Identify any relevant topics that relate to family influence on substance use, such as parenting style, parental attitudes towards substances, family conflicts, or parental substance use patterns. If the document reflects family-related themes, output '[1] Family Influence' and briefly describe the relevant family-related context in the document. Otherwise, if no family influence is observed, return 'None'.\n\nDocument:\n{post_content}"
        },
        {
            "topic": "Peer Influence",
            "prompt": f"You will receive a document related to adolescent behaviors and substance use. Identify any relevant topics that indicate peer influence, such as peer pressure, social exclusion, group behavior at gatherings, or friends' substance use behaviors. If peer influence themes are present, output '[1] Peer Influence' with a brief description of the relevant peer-related context. If not, return 'None'.\n\nDocument:\n{post_content}"
        },
        {
            "topic": "School Environment",
            "prompt": f"You will receive a document related to adolescent behaviors and substance use. Identify any relevant topics associated with the school environment, including school culture, teacher-student relationships, academic pressure, and student satisfaction with school life. If school-related factors are mentioned, output '[1] School Environment' with a brief description. If not, return 'None'.\n\nDocument:\n{post_content}"
        }
    ]

    results = {}
    for prompt_info in prompts:
        try:
            response = openai.chat.completions.create(
               messages=[
                {"role": "system", "content": prompt_info["prompt"] },
               
                    ],
                model="gpt-4o-mini",  # Adjust model if necessary
                
                max_tokens=50,
                temperature=0.5
            )
            result_text = response.choices[0].message.content.strip().lower()
            results[prompt_info["topic"]] = result_text if result_text != 'None' else None
        except Exception as e:
            print(f"An error occurred: {e}")
            results[prompt_info["topic"]] = None

    return results



In [31]:
sampledf=substance_use_df.head(5)

In [ ]:
from tqdm import tqdm
tqdm.pandas()
# Apply the analysis function to each post
sampledf[['Family Influence', 'Peer Influence', 'School Environment']] = sampledf['posts'].progress_apply(
    lambda post: pd.Series(analyze_post(post))
)

# Display or save the results
print(sampledf[['posts', 'Family Influence', 'Peer Influence', 'School Environment']])

In [41]:
from tqdm import tqdm
tqdm.pandas()
# Apply the analysis function to each post
substance_use_df[['Family Influence', 'Peer Influence', 'School Environment']] = substance_use_df['posts'].progress_apply(
    lambda post: pd.Series(analyze_post(post))
)

100%|██████████| 21169/21169 [12:57:16<00:00,  2.20s/it]  


In [ ]:
substance_use_df.to_csv('',index=False)

In [ ]:
substance_use_df =pd.read_csv('')

In [25]:
substance_use_df.head(3)

,author,created_utc,selftext,title,subreddit,date,posts,emotion,substance,Family Influence,Peer Influence,School Environment
0,DylSan_07,2021-10-01 01:58:35,many people got searched and there were rumors...,A kid got caught doing cocaine in the school b...,teenagers,2021-10-01 01:58:35,kid got caught cocaine school bathrooms 26 peo...,disgust,1,none,[1] peer influence: the document highlights a ...,[1] school environment - the document mentions...
1,SheldorBackOnline5,2021-10-01 02:58:09,every time i go out with my friends it’s alway...,My mother doesn’t trust me,teenagers,2021-10-01 02:58:09,mother ’ trust every time go friends ’ always ...,neutral,1,[1] family influence: the document reflects th...,[1] peer influence: the document discusses the...,none
2,Beneficial-Egg-3445,2021-10-01 03:52:43,i'm in a hotel room with my grandparents for a...,Please help.,teenagers,2021-10-01 03:52:43,please help m hotel room grandparents wedding ...,disgust,1,none,none,none


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming `df` is your DataFrame with an `emotion` column

# Calculate the distribution of emotions
emotion_counts = substance_use_df['emotion'].value_counts()
emotion_percentages = (emotion_counts / len(substance_use_df)) * 100  # Calculate percentages

# Print the emotion distribution percentages
print("\nEmotion Distribution (Percentage):")
print(emotion_percentages)

# Plot a pie chart for emotion distribution
plt.figure(figsize=(8, 6))
plt.pie(emotion_percentages, labels=emotion_percentages.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
plt.title("Percentage Distribution of Emotions")
plt.show()

# Define a custom color palette for each emotion in the bar chart
custom_colors = ["#FFA07A", "#87CEFA", "#FF6347", "#6495ED", "#FFD700", "#32CD32", "#BA55D3"]  # A mix of orange, blue, and other colors

# Plot a bar chart for emotion distribution with different colors
plt.figure(figsize=(10, 6))
emotion_percentages.plot(kind='bar', color=custom_colors[:len(emotion_percentages)])  # Use as many colors as there are emotions

plt.title("Percentage Distribution of Emotions")
plt.xlabel("Emotion")
plt.ylabel("Percentage")
plt.show()